## Data Transformation Trial-runs

In [ ]:
import os 

In [ ]:
os.chdir()

## Entity Trial-runs  

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

## ConfigurationManager Trial-runs 

In [ ]:
#importing all project paths and modules necessary for project configurations 
from textSummerizer.constants import  *
from textSummerizer.utils.common import read_yaml, create_directories

In [ ]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation 
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path, 
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

## Components Trial-runs 

In [ ]:
import os 
from textSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [ ]:
class DataTransformation: 
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings =self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'], 
        'labels': target_encodings['input_ids']
    }

    def convert(self): 
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_example_to_features, batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
        
    

## Pipeline trial-runs

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config =config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e